In [1]:
%pip install python-dotenv pandas plotly requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import requests
import time
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go

In [3]:

load_dotenv('../.env')
API_KEY = os.getenv('PUBLIC_API_KEY')

BASE_URL = f'http://openapi.seoul.go.kr:8088/756e76706a68666f3131377a57717353/json/tbLnOpendataRtmsV'

years = [2023, 2024, 2025]
districts = {
    '서초구': '11650',
    '강남구': '11680',
    '송파구': '11710'
}

rows_all = []
step = 1000

for year in years:
    for gu_nm, gu_cd in districts.items():
        start = 1

        while True:
            end = start + step - 1
            url = f'{BASE_URL}/{start}/{end}/{year}/{gu_cd}'

            res = requests.get(url)
            if res.status_code != 200:
                print('요청 실패:', year, gu_nm)
                break

            data = res.json().get('tbLnOpendataRtmsV', {})
            rows = data.get('row', [])

            if not rows:
                break

            rows_all.extend(rows)
            print(f'{year} {gu_nm} {start}~{end} 수집')

            start += step
            time.sleep(2)

df = pd.DataFrame(rows_all)

if df.empty:
    print("데이터가 없습니다.")
else:
    df = df[df['BLDG_USG'] == '아파트']

    df = df[
        [
            'RCPT_YR',        # 접수연도
            'CGG_CD',         # 자치구 코드
            'CGG_NM',         # 자치구명
            'STDG_CD',        # 법정동 코드
            'STDG_NM',        # 법정동 명
            'BLDG_NM',        # 건물명
            'CTRT_DAY',       # 계약일
            'THING_AMT',	  # 물건금액(만원)
            'ARCH_AREA',	  # 건물면적(㎡)
            'LAND_AREA',	  # 토지면적(㎡)
            'FLR',	          # 층
            'ARCH_YR',        # 건축년도
            'BLDG_USG',       # 건물용도
        ]
    ]

    # 계약일자 기준 필터링 추가
    # CTRT_DAY를 datetime으로 변환
    # 거래는 2022-12-31에 했으나 신고를 2023년에 하는 케이스가 존재
    df['CTRT_DAY'] = pd.to_datetime(df['CTRT_DAY'], format='%Y%m%d', errors='coerce')
    
    # 2023-01-01 ~ 2025-12-31 사이만 필터링
    df = df[
        (df['CTRT_DAY'] >= '2023-01-01') & 
        (df['CTRT_DAY'] <= '2025-12-31')
    ]

    # 중복 제거
    print(f"\n중복 제거 전: {len(df):,}건")
    
    df_clean = df.drop_duplicates(subset=[
        'CTRT_DAY',
        'BLDG_NM', 
        'ARCH_AREA',
        'FLR',
        'THING_AMT'
    ], keep='first')

    print(f"중복 제거 후: {len(df_clean):,}건")

    # CTRT_DAY를 다시 문자열로 변환 (저장 시 형식 유지)
    df_clean['CTRT_DAY'] = df_clean['CTRT_DAY'].dt.strftime('%Y%m%d')

    df_clean.to_csv(
        '../data/seoul_apartment_2023_2025_gangnam.csv',
        index=False,
        encoding='utf-8-sig'
    )

    display(df.head())

2023 서초구 1~1000 수집
2023 서초구 1001~2000 수집
2023 서초구 2001~3000 수집
2023 강남구 1~1000 수집
2023 강남구 1001~2000 수집
2023 강남구 2001~3000 수집
2023 강남구 3001~4000 수집
2023 송파구 1~1000 수집
2023 송파구 1001~2000 수집
2023 송파구 2001~3000 수집
2023 송파구 3001~4000 수집
2023 송파구 4001~5000 수집
2024 서초구 1~1000 수집
2024 서초구 1001~2000 수집
2024 서초구 2001~3000 수집
2024 서초구 3001~4000 수집
2024 서초구 4001~5000 수집
2024 강남구 1~1000 수집
2024 강남구 1001~2000 수집
2024 강남구 2001~3000 수집
2024 강남구 3001~4000 수집
2024 강남구 4001~5000 수집
2024 강남구 5001~6000 수집
2024 송파구 1~1000 수집
2024 송파구 1001~2000 수집
2024 송파구 2001~3000 수집
2024 송파구 3001~4000 수집
2024 송파구 4001~5000 수집
2024 송파구 5001~6000 수집
2024 송파구 6001~7000 수집
2025 서초구 1~1000 수집
2025 서초구 1001~2000 수집
2025 서초구 2001~3000 수집
2025 서초구 3001~4000 수집
2025 서초구 4001~5000 수집
2025 강남구 1~1000 수집
2025 강남구 1001~2000 수집
2025 강남구 2001~3000 수집
2025 강남구 3001~4000 수집
2025 강남구 4001~5000 수집
2025 강남구 5001~6000 수집
2025 강남구 6001~7000 수집
2025 송파구 1~1000 수집
2025 송파구 1001~2000 수집
2025 송파구 2001~3000 수집
2025 송파구 3001~4000 수집
2025 송파구 4001~5

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15556\2030547868.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['CTRT_DAY'] = df_clean['CTRT_DAY'].dt.strftime('%Y%m%d')


,RCPT_YR,CGG_CD,CGG_NM,STDG_CD,STDG_NM,BLDG_NM,CTRT_DAY,THING_AMT,ARCH_AREA,LAND_AREA,FLR,ARCH_YR,BLDG_USG
1,2023,11650,서초구,10800,서초동,서초1차e-편한세상,2023-12-28,220000,130.53,0.0,4.0,2004,아파트
4,2023,11650,서초구,10300,우면동,서초힐스,2023-12-27,123000,74.97,0.0,8.0,2012,아파트
5,2023,11650,서초구,10700,반포동,반포파크빌,2023-12-27,175000,110.79,0.0,4.0,2002,아파트
7,2023,11650,서초구,10100,방배동,SK리더스뷰(파스텔시티),2023-12-26,140000,84.95,0.0,13.0,2006,아파트
13,2023,11650,서초구,10300,우면동,서초힐스,2023-12-22,119000,59.89,0.0,15.0,2012,아파트


In [4]:
# 저장된 csv 읽기
df = pd.read_csv('../data/seoul_apartment_2023_2025_gangnam.csv', encoding='utf-8-sig')

In [5]:
# 중복값 확인 (거래일 + 건물명 + 면적 + 층 + 가격)
duplication = df[df.duplicated(subset=[
    'CTRT_DAY',  # 계약일
    'BLDG_NM',   # 건물명
    'ARCH_AREA', # 건물면적(㎡)
    # 'LAND_AREA', # 토지면적(㎡)
    'FLR',       # 층
    'THING_AMT'  # 물건금액(만원)
], keep='first')]

print(f"중복 건수: {len(duplication)}건")
display(duplication.head())

중복 건수: 0건


,RCPT_YR,CGG_CD,CGG_NM,STDG_CD,STDG_NM,BLDG_NM,CTRT_DAY,THING_AMT,ARCH_AREA,LAND_AREA,FLR,ARCH_YR,BLDG_USG


In [6]:
# # 컬럼 정보
# display(df.info())

# # 상위 5개 행 확인
# display(df.head())

# # 결측치 확인
# display(df.isna().sum()) # 전체 확인

# # 요약 통계
# display(df.describe())

# df['price_per_sqm'].describe()

In [7]:
# 전처리 및 파생변수 생성

# 날짜 변환
df['CTRT_DAY'] = pd.to_datetime(df['CTRT_DAY'], format='%Y%m%d')

# 년월 컬럼 생성
df['year_month'] = df['CTRT_DAY'].dt.to_period('M').astype(str)
# df['year_month'] = df['CTRT_DAY'].dt.to_period('M')

# ㎡당 가격 (만원)
df['price_per_sqm'] = df['THING_AMT'] / df['ARCH_AREA']


Q1 = df['price_per_sqm'].quantile(0.25)
Q3 = df['price_per_sqm'].quantile(0.75)
IQR = Q3 - Q1

outliers = df[
    (df['price_per_sqm'] < Q1 - 1.5*IQR) | 
    (df['price_per_sqm'] > Q3 + 1.5*IQR)
]

print(f"전체: {len(df):,}건")
print(f"이상치: {len(outliers):,}건")
print(f"비율: {len(outliers)/len(df)*100:.1f}%")

# 1. 래미안 원베일리: 98건 (평균 57억)
# 2. 아크로리버파크: 56건 (평균 58억)
# 3. PH129: 2건 (평균 172억!)
# 4. 한양1차: 20건 (평균 ㎡당 7,104만원)

# → 전부 실제 거래!
# → 데이터 오류 아님!
# 제거하면 시장 왜곡

# 규모 구분
def size_group(area):
    if area <= 60: # 18.15평
        return '소형'
    elif area <= 85: # 25.7평
        return '중형'
    else:
        return '대형'

df['size_group'] = df['ARCH_AREA'].apply(size_group)

df.head()


전체: 28,315건
이상치: 491건
비율: 1.7%


,RCPT_YR,CGG_CD,CGG_NM,STDG_CD,STDG_NM,BLDG_NM,CTRT_DAY,THING_AMT,ARCH_AREA,LAND_AREA,FLR,ARCH_YR,BLDG_USG,year_month,price_per_sqm,size_group
0,2023,11650,서초구,10800,서초동,서초1차e-편한세상,2023-12-28,220000,130.53,0.0,4.0,2004,아파트,2023-12,1685.436298,대형
1,2023,11650,서초구,10300,우면동,서초힐스,2023-12-27,123000,74.97,0.0,8.0,2012,아파트,2023-12,1640.656263,중형
2,2023,11650,서초구,10700,반포동,반포파크빌,2023-12-27,175000,110.79,0.0,4.0,2002,아파트,2023-12,1579.564943,대형
3,2023,11650,서초구,10100,방배동,SK리더스뷰(파스텔시티),2023-12-26,140000,84.95,0.0,13.0,2006,아파트,2023-12,1648.028252,중형
4,2023,11650,서초구,10300,우면동,서초힐스,2023-12-22,119000,59.89,0.0,15.0,2012,아파트,2023-12,1986.976123,소형


In [8]:
# 구별 월 평균(핵심 지표)

monthly_gu = (
    df.groupby(['year_month', 'CGG_NM'])
      .agg(
          avg_price_per_sqm=('price_per_sqm', 'mean'), # ㎡당가격
          median_price_per_sqm=('price_per_sqm', 'median'), # 이상치 처리
          avg_price=('THING_AMT', 'mean'),             # 총가격
          txn_cnt=('THING_AMT', 'count')               # 거래량
      )
      .reset_index()
)

monthly_gu.head()

,year_month,CGG_NM,avg_price_per_sqm,median_price_per_sqm,avg_price,txn_cnt
0,2023-01,강남구,2451.853119,2472.624514,183800.756757,111
1,2023-01,서초구,2483.022641,2345.038888,240656.603774,53
2,2023-01,송파구,1963.543839,2027.226970,156313.000000,150
3,2023-02,강남구,2412.021795,2591.483266,177189.202073,193
4,2023-02,서초구,2453.184221,2472.109977,217119.767442,86


In [9]:
# 규모별 월 평균 (해석력 높히기)
monthly_size = (
    df.groupby(['year_month', 'CGG_NM', 'size_group'])
      .agg(
          avg_price_per_sqm=('price_per_sqm', 'mean'), # ㎡당가격
          txn_cnt=('THING_AMT', 'count')               # 거래량
      )
      .reset_index()
)

monthly_size.head()

,year_month,CGG_NM,size_group,avg_price_per_sqm,txn_cnt
0,2023-01,강남구,대형,2207.101153,22
1,2023-01,강남구,소형,2476.885247,48
2,2023-01,강남구,중형,2553.877537,41
3,2023-01,서초구,대형,2324.681900,18
4,2023-01,서초구,소형,2285.214130,14


In [10]:
# 구별 누적 상승률

price_start_end = (
    monthly_gu
    .sort_values('year_month')
    .groupby('CGG_NM')
    .agg(
        start_price=('avg_price_per_sqm', 'first'),  # 시작가격
        end_price=('avg_price_per_sqm', 'last'),     # 종료가격
        start_median=('median_price_per_sqm', 'first'),
        end_median=('median_price_per_sqm', 'last')
    )
)


# 변화량
price_start_end['change_amt'] = (
    price_start_end['end_price'] - price_start_end['start_price']
)

price_start_end['change_pct'] = (
    price_start_end['change_amt'] / price_start_end['start_price'] * 100
)

# 증가율
price_start_end['change_pct_median'] = (
    (price_start_end['end_median'] - price_start_end['start_median']) / 
    price_start_end['start_median'] * 100
)

display(price_start_end)

# change_pct(평균) / change_pct_median(중긴값)
display(price_start_end[['change_pct', 'change_pct_median']])

,start_price,end_price,start_median,end_median,change_amt,change_pct,change_pct_median
CGG_NM,,,,,,,
강남구,2451.853119,3230.156363,2472.624514,3119.074566,778.303244,31.743469,26.144287
서초구,2483.022641,2126.895180,2345.038888,1890.833417,-356.127461,-14.342498,-19.368782
송파구,1963.543839,2307.374709,2027.226970,2017.342532,343.830870,17.510730,-0.487584


,change_pct,change_pct_median
CGG_NM,,
강남구,31.743469,26.144287
서초구,-14.342498,-19.368782
송파구,17.510730,-0.487584


In [11]:
# 1. 구별 월별 평균 가격 추이
fig1 = px.line(
    monthly_gu,
    x='year_month',
    y='avg_price',
    color='CGG_NM',
    title='강남 3구 월별 평균 실거래가 추이 (2023-2025)',
    labels={
        'year_month': '년월',
        'avg_price': '평균 실거래가 (만원)',
        'CGG_NM': '구'
    },
    markers=True,
    color_discrete_map={
        '강남구': '#FF6B6B',
        '서초구': '#4ECDC4',
        '송파구': '#95E1D3'
    }
)

fig1.update_layout(
    hovermode='x unified',
    xaxis_tickangle=-45,
    height=600
)

fig1.show()

In [12]:
# 2. 구별 증가율 비교
fig2 = px.bar(
    price_start_end.reset_index(),
    x='CGG_NM',
    y='change_pct',
    title='구별 가격 증가율 (2023-01 대비 2025-12)',
    labels={
        'CGG_NM': '구',
        'change_pct': '증가율 (%)'
    },
    text='change_pct',
    color='change_pct',
    color_continuous_scale='RdYlGn'
)

fig2.update_traces(
    texttemplate='%{text:.1f}%',
    textposition='outside'
)

fig2.update_layout(height=500)
fig2.show()

print('='*80)
print("구별 증가율")
print("="*80)

for gu in price_start_end.index:
    mean_pct = price_start_end.loc[gu, 'change_pct']
    median_pct = price_start_end.loc[gu, 'change_pct_median']
    
    print(f"\n{gu}:")
    print(f"  평균:   {mean_pct:+.1f}%")
    print(f"  중간값: {median_pct:+.1f}%")

구별 증가율

강남구:
  평균:   +31.7%
  중간값: +26.1%

서초구:
  평균:   -14.3%
  중간값: -19.4%

송파구:
  평균:   +17.5%
  중간값: -0.5%


In [13]:
# 3. 규모별 ㎡당 가격 추이
fig3 = px.line(
    monthly_size,
    x='year_month',
    y='avg_price_per_sqm',
    color='size_group',
    facet_col='CGG_NM',  # 구별로 분할
    title='구별·규모별 ㎡당 평균 가격 추이',
    labels={
        'year_month': '년월',
        'avg_price_per_sqm': '㎡당 가격 (만원)',
        'size_group': '규모',
        'CGG_NM': '자치구'
    },
    markers=True
)

fig3.update_xaxes(tickangle=-45)
fig3.update_layout(height=500)
fig3.show()

In [14]:
# 기존 monthly_gu에서 거래량만 피벗
txn_pivot = monthly_gu.pivot(
    index='year_month',
    columns='CGG_NM',
    values='txn_cnt'
).reset_index()

display(txn_pivot)  # .head() 대신 전체 출력

CGG_NM,year_month,강남구,서초구,송파구
0,2023-01,111,53,150
1,2023-02,193,86,256
2,2023-03,182,122,232
3,2023-04,197,151,279
4,2023-05,265,153,297
5,2023-06,277,184,290
6,2023-07,252,199,269
7,2023-08,281,198,266
8,2023-09,201,148,259
9,2023-10,143,90,146


In [15]:
# 4. 거래량과 가격 동시 표시 (듀얼 축)
fig4 = go.Figure()

# 강남구 예시
gangnam = monthly_gu[monthly_gu['CGG_NM'] == '강남구']

# 가격 (왼쪽 축)
fig4.add_trace(go.Scatter(
    x=gangnam['year_month'],
    y=gangnam['avg_price'],
    name='평균 가격',
    yaxis='y',
    mode='lines+markers',
    line=dict(color='#FF6B6B', width=3)
))

# 거래량 (오른쪽 축)
fig4.add_trace(go.Bar(
    x=gangnam['year_month'],
    y=gangnam['txn_cnt'],
    name='거래량',
    yaxis='y2',
    opacity=0.3,
    marker_color='#95E1D3'
))

fig4.update_layout(
    title='강남구 가격 vs 거래량 추이',
    xaxis=dict(title='년월', tickangle=-45),
    yaxis=dict(
        title='평균 가격 (만원)',
        side='left'
    ),
    yaxis2=dict(
        title='거래량 (건)',
        side='right',
        overlaying='y'
    ),
    hovermode='x unified',
    height=600
)

fig4.show()

In [16]:
# 서초구 가격 vs 거래량
fig_seocho = go.Figure()

seocho = monthly_gu[monthly_gu['CGG_NM'] == '서초구']

fig_seocho.add_trace(go.Scatter(
    x=seocho['year_month'],
    y=seocho['avg_price'],
    name='평균 가격',
    yaxis='y',
    mode='lines+markers',
    line=dict(color='#4ECDC4', width=3)
))

fig_seocho.add_trace(go.Bar(
    x=seocho['year_month'],
    y=seocho['txn_cnt'],
    name='거래량',
    yaxis='y2',
    opacity=0.3,
    marker_color='#95E1D3'
))

fig_seocho.update_layout(
    title='서초구 가격 vs 거래량 추이',
    xaxis=dict(title='년월', tickangle=-45),
    yaxis=dict(
        title='평균 가격 (만원)',
        side='left'
    ),
    yaxis2=dict(
        title='거래량 (건)',
        side='right',
        overlaying='y'
    ),
    hovermode='x unified',
    height=600
)

fig_seocho.show()

In [17]:
# 송파구 가격 vs 거래량
fig_songpa = go.Figure()

songpa = monthly_gu[monthly_gu['CGG_NM'] == '송파구']

fig_songpa.add_trace(go.Scatter(
    x=songpa['year_month'],
    y=songpa['avg_price'],
    name='평균 가격',
    yaxis='y',
    mode='lines+markers',
    line=dict(color='#95E1D3', width=3)
))

fig_songpa.add_trace(go.Bar(
    x=songpa['year_month'],
    y=songpa['txn_cnt'],
    name='거래량',
    yaxis='y2',
    opacity=0.3,
    marker_color='#FF6B6B'
))

fig_songpa.update_layout(
    title='송파구 가격 vs 거래량 추이',
    xaxis=dict(title='년월', tickangle=-45),
    yaxis=dict(
        title='평균 가격 (만원)',
        side='left'
    ),
    yaxis2=dict(
        title='거래량 (건)',
        side='right',
        overlaying='y'
    ),
    hovermode='x unified',
    height=600
)

fig_songpa.show()

In [18]:
# 5. 구별 가격 분포 (박스플롯)
fig5 = px.box(
    df,
    x='CGG_NM',
    y='THING_AMT',
    color='size_group',
    title='구별·규모별 가격 분포',
    labels={
        'CGG_NM': '구',
        'THING_AMT': '거래가 (만원)',
        'size_group': '규모'
    },
    points='outliers'  # 이상치만 점으로 표시
)

fig5.update_layout(height=600)
fig5.show()

In [19]:
# 6. 히트맵 (월별 x 구별 가격)
pivot_data = monthly_gu.pivot(
    index='CGG_NM',
    columns='year_month',
    values='avg_price'
)

fig6 = px.imshow(
    pivot_data,
    title='월별·구별 평균 가격 히트맵',
    labels=dict(x='년월', y='구', color='평균가 (만원)'),
    color_continuous_scale='YlOrRd',
    aspect='auto'
)

fig6.update_xaxes(tickangle=-45)
fig6.update_layout(height=500)
fig6.show()